## Aperture測光の方法

#### パッケージのimport

In [1]:
import astropy as ap # astropyのimport
import astropy.io.fits
import photutils as phot # 測光のためのパッケージのimport
import numpy as np

#### データの読み込み

In [2]:
im = "/media/dionysos1/data/public/Transit/GJ-436/170427/L0131000.fits" # FITSファイルの名前
hdu = ap.io.fits.open(im) # FITSファイルの読み込み
header = hdu[0].header # header部分の取り出し
im = hdu[0].data # data部分の取り出し
gain = header["GAIN"] # header内のGAIN値の読み込み
ron = header["RONOISE"] #Read Out NOISE値の読み込み

#### aperture photometry

In [3]:
# aperture photometryのための設定
pos = [938,1097] # apertureの円中心(x,y)の座標(画像上のピクセル座標)
r = 50 # apertureの半径
r_in = 65 # 円環の内側の半径
r_out = 75 # 円環の外側の半径

apertures = phot.CircularAperture(pos, r) # 様々な半径radiiの円のaperture領域を定義
sky_annulus = phot.CircularAnnulus(pos, r_in=r_in, r_out=r_out) # 円環のaperture領域を定義
phot_table = phot.aperture_photometry(im, apertures) # 恒星のaperture合計をとる。
sky_table = phot.aperture_photometry(im, sky_annulus) # skyのring状のaperture合計をとる。
bkg_mean = sky_table["aperture_sum"] / sky_annulus.area() # skyの1pixelあたりの平均値(# より精度の良い解析のためには、skyをmedianで計算した方がよい)
bkg_sum = bkg_mean * apertures.area() * gain # 1ピクセルあたりのsky値をaperture半径内のピクセルにかけて、aperture内のsky値の総数を計算
final_sum = gain * phot_table["aperture_sum"] - bkg_sum # aperture測光した値から、そのaperture半径分のピクセルのskyを引く,これはgainをかけているのでelectron数
err = np.sqrt(final_sum + ron*ron* apertures.area() + bkg_sum)  # エラー値

print(final_sum)
print(err)

   aperture_sum   
------------------
16592835.767128758
   aperture_sum   
------------------
6671.3138363782755
